In [1]:
import torch
import torch.nn as nn #import neural network class
from torch.utils.data import DataLoader #helps in creating batchs
from torchvision import datasets #download data
from torchvision.transforms import ToTensor
import torch.nn.functional as F

In [2]:
## download data
training_data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
test_data = datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

100%|██████████| 26421880/26421880 [00:01<00:00, 15903806.43it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 259788.93it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5051521.21it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5940103.71it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size) #inputdata and Outputdata
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [4]:
for X,y in train_dataloader:
  print (X.shape)
  print (y.shape)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
len(train_dataloader)

938

In [6]:
len(test_dataloader)

157

In [7]:
## device setup

device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
device

'cuda'

In [9]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__() ##it copies the initializations from the nn.Module class
    self.flatten = nn.Flatten() #28x28x1 is converted to 764x1
    self.linear1 = nn.Linear(28*28,512)
    self.linear2 = nn.Linear(512,512)
    self.output = nn.Linear(512,10) #10 class classification problem

  def forward(self,x):
    x = self.flatten(x)
    x = self.linear1(x)
    x = F.relu(x)
    x = self.linear2(x)
    x = F.relu(x)
    out = self.output(x) #We didn't use softmax here
    return out

In [10]:
model = NeuralNetwork().to(device)
print (model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=10, bias=True)
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.0001)

In [46]:
## define the training loop

def train(dataloader,model,loss_fn,optimizer):
  model.train()
  for batch,(X,y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)

    pred = model(X) #stores the variables required for gradient descent
    loss = loss_fn(pred,y)

    ##backpropogation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      print (f'Loss : {loss.item()}')

In [55]:
def test(dataloader,model,loss_fn):
  model.eval() #it is in test mode
  test_loss, correct = 0,0
  size = len(dataloader.dataset)
  with torch.no_grad(): #What ever is inside this steps no gradient should be applied
    for batch,(X,y) in enumerate(dataloader):
      X,y = X.to(device), y.to(device)
      pred = model(X) #10x1 vector
      # print (pred.shape)
      # print (pred.argmax(1))
      # print (y)
      # print ((pred.argmax(1) == y).type(torch.float).sum().item())
      # break
      test_loss += loss_fn(pred,y).item()
      '''
      pred.argmax(1) -- position of the maximum probability
      torch.float
      torch.int
      torch.float32
      '''
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  print (f'Total correct {correct} out of {size}')
  Accuracy = correct/size
  print (f'Accuracy : {Accuracy*100}')

In [56]:
epochs = 5

for t in range(epochs):
  print ('Epoch---------------------------')
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader, model, loss_fn)

Epoch---------------------------
Loss : 2.086418867111206
Loss : 2.0952680110931396
Loss : 2.054330587387085
Loss : 2.1035802364349365
Loss : 2.0787112712860107
Loss : 2.02934193611145
Loss : 2.0934078693389893
Loss : 2.0412981510162354
Loss : 2.071528673171997
Loss : 2.038721799850464
Total correct 4758.0 out of 10000
Accuracy : 47.58
Epoch---------------------------
Loss : 2.0638632774353027
Loss : 2.0723419189453125
Loss : 2.0277509689331055
Loss : 2.080895185470581
Loss : 2.052708864212036
Loss : 2.0022778511047363
Loss : 2.0692598819732666
Loss : 2.0137622356414795
Loss : 2.0465035438537598
Loss : 2.0115609169006348
Total correct 4895.0 out of 10000
Accuracy : 48.949999999999996
Epoch---------------------------
Loss : 2.0398032665252686
Loss : 2.04766583442688
Loss : 1.9992715120315552
Loss : 2.0564358234405518
Loss : 2.0246593952178955
Loss : 1.9735620021820068
Loss : 2.0431318283081055
Loss : 1.9843323230743408
Loss : 2.0197834968566895
Loss : 1.9825905561447144
Total correct 49

In [57]:
!pwd

/content


In [58]:
## save the model
torch.save(model.state_dict(),"/content/Myfirstmodel.pth")

In [59]:
## Load t
model = NeuralNetwork().to(device)
#torch.load_state_dict(torch.load('/content/Myfirstmodel.pth'))

AttributeError: ignored